# Usando o OSMnx para obter dados de SIG

## 1. Analizando a origem da água em Florianópolis

<div class="alert alert-info" style="font-size:100%">

Quando prédios existem na área metropolitana? De onde é oriunda a água que abastece a região?

</div>

In [ ]:
lon, lat = -48.548889, -27.596944

dx = dy = 0.05

bbox = [lon-dx, lat+dy, lon+dx, lat-dy]
west, north, east, south = bbox

bbox_ctr = [0.5*(north+south), 0.5*(west+east)]

bbox

## 2. OSMnx extrai informações da base de dados do OpenStreetMaps

<div class="alert alert-info" style="font-size:100%">

Veja https://github.com/gboeing/osmnx-examples/blob/master/notebooks/10-building-footprints.ipynb e https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.footprints
para entender mais como o OSMnx funciona.

</div>

In [ ]:
import osmnx


osmnx.config(use_cache=True, log_console=True)

buildings = osmnx.create_footprints_gdf(
    north=north,
    south=south,
    east=east,
    west=west,
    footprint_type="building",
)

In [ ]:
buildings.crs

In [ ]:
len(buildings)

In [ ]:
%matplotlib inline


buildings.plot(edgecolor="grey", linewidth=2);

In [ ]:
buildings.geom_type.unique()

In [ ]:
buildings.columns

In [ ]:
buildings.head()

### Explore a distribuição de valores de algumas dessas colunas.

In [ ]:
buildings["amenity"].value_counts()

```
buildings["building"].value_counts()
```

### Re-projetar para UMT para calcular áreas

Veja [como achar o CRS](https://gis.stackexchange.com/questions/190198/how-to-get-appropriate-crs-for-a-position-specified-in-lat-lon-coordinates) ane [a UTM zone](https://gis.stackexchange.com/questions/269518/auto-select-suitable-utm-zone-based-on-grid-intersection) via stackexchange.

In [ ]:
import utm


utmzone_code = utm.from_latlon(bbox_ctr[0], bbox_ctr[1])[-2:]
utmzone_epsg = 32700 - round((45+bbox_ctr[0])/90)*100 + round((183+bbox_ctr[1])/6)
utmzone_code, utmzone_epsg

In [ ]:
buildings_utm = buildings.to_crs(epsg=utmzone_epsg)

In [ ]:
building_area_m2 = buildings_utm.geometry.area

In [ ]:
building_area_m2.min(), building_area_m2.mean(), building_area_m2.median(), building_area_m2.max()

In [ ]:
buildings.loc[building_area_m2 == building_area_m2.max()]

In [ ]:
bins_seq = [k for k in range(0, 10000, 200)]
ax = buildings_utm.geometry.area.hist(bins=bins_seq)
ax.set_xlabel("Pegada dos prédios de Floripa (m$^2$)");

## 3. Extrair os "waterways" usando `graph_from_bbox`

Ceque [esse notebook exemplo fo osmnx](https://github.com/gboeing/osmnx-examples/blob/master/notebooks/16-work-with-other-infrastructures.ipynb) para maiores informações.

<div class="alert alert-info" style="font-size:100%">
<b style="font-size:120%">Navegando nos tipos de tag do OSM</b></br>

OSM ["representa feições físicas (ex., estradas ou prédios) usando tags nas suas estruturas de dados básicos (seus nós, vias, e relações)"](https://wiki.openstreetmap.org/wiki/Map_Features). Essas tags podem ser complicadas de navegar. Vá para https://wiki.openstreetmap.org/wiki/Map_Features para navegar pelas tags de categoria _overlapping_. _Waterways_ são encontradas em várias formas; as mais úties são aqui _Waterway_ and _Natural_.

</div>

### Lendo as "vias" de `waterway`

In [ ]:
waterways = osmnx.graph_from_bbox(
    north=north,
    south=south,
    east=east,
    west=west,
    network_type="none",
    infrastructure="way['waterway']",
    retain_all=True,
    truncate_by_edge=True,
    simplify=False,
)

In [ ]:
waterways_edges = osmnx.graph_to_gdfs(waterways, nodes=False)

len(waterways_edges)

In [ ]:
waterways_edges.head()

In [ ]:
waterways_edges.plot();

### Lendo as vias de  `natural:water`

In [ ]:
natwater = osmnx.graph_from_bbox(
    north=north,
    south=south,
    east=east,
    west=west,
    network_type="none",
    infrastructure="way['natural'~'water']",
    retain_all=True,
    truncate_by_edge=True,
    simplify=False,
)

natwater_edges = osmnx.graph_to_gdfs(natwater, nodes=False)
len(natwater_edges)

In [ ]:
natwater_edges.plot();

## 4. Colocando tudo junto!

In [ ]:
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(10, 10))

waterways_edges.plot(ax=ax, edgecolor="lightblue", linewidth=1)
natwater_edges.plot(ax=ax, edgecolor="darkblue", linewidth=3)
buildings.plot(ax=ax, edgecolor="grey", linewidth=2, alpha=0.25);

In [ ]:
%load exe04.py

<div class="alert alert-warning" style="font-size:100%">

Desafios:

1. Gere um mapa semelhante para a reginão metropolitana de sua cidade ou cidade de interesse.
2. Faça o gráfico final em um mapa _slippy_.

</div>

<div class="alert alert-success" style="font-size:100%">

Utilizando apenas uma biblioteca, OSMnx, conseguimos (quase! falta a análise) gerar um mapa de sensibilidade de abastecimento de água para a região metropolitana de Florianólis.

</div>